### Extract 100 time series, each with 100 data points, from PdM_telemetry.csv.

To ensure diversity, we used different attributes for specific groups of machines: the "voltage" attribute for machines 1-25, "rotation" for machines 26-50, "pressure" for machines 51-75, and "vibration" for machines 76-100. This process resulted in 100 distinct time series, each containing 100 time points.

In [1]:
import pandas as pd

# Read in csv file
file_path = "PdM_telemetry.csv" 
df = pd.read_csv(file_path)

df['datetime'] = pd.to_datetime(df['datetime'])

time_series_data = {}

for machine_id in range(1, 101):
    machine_data = df[df['machineID'] == machine_id].sort_values(by="datetime")
    
    # select 900-1000 data points
    selected_data = machine_data.iloc[900:1000]
    # select first 1000 data points
    # selected_data = machine_data.iloc[:1000] # for Chronos
    
    # Different variable column for each machine group
    if 1 <= machine_id <= 25:
        attribute = "volt"
    elif 26 <= machine_id <= 50:
        attribute = "rotate"
    elif 51 <= machine_id <= 75:
        attribute = "pressure"
    else:
        attribute = "vibration"
    
    time_series = selected_data[['datetime', attribute]].reset_index(drop=True)
    time_series[attribute] = time_series[attribute].round(2)
    time_series_data[machine_id] = time_series

# for machine_id, ts_data in time_series_data.items():
#     ts_data.to_csv(f"PdM_time_series_machine_{machine_id}.csv", index=False)

# write 100 time series into a single excel with 100 sheets
with pd.ExcelWriter("PdM_machine_time_series.xlsx") as writer:
# with pd.ExcelWriter("Chronos_PdM_machine_time_series.xlsx") as writer:
    for machine_id, ts_data in time_series_data.items():
        ts_data.to_excel(writer, sheet_name=f"Machine_{machine_id}", index=False)

print("100 time series extracted successfully.")


/var/folders/fh/x42m5xbn25d25fy3bskk3jkm0000gn/T/ipykernel_81105/3846124247.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['datetime'] = pd.to_datetime(df['datetime'])


100 time series extracted successfully.


### 100 time series for slope calculation task, trend & noise need to be added to the extracted data.

In [ ]:
## Positive slope

import pandas as pd

# Define transformation parameters
trend = 2
noise = 6

# Load the Excel file
# excel_file = "PdM_machine_time_series.xlsx"
excel_file = "Chronos_PdM_machine_time_series.xlsx" # for Chronos

# Read all sheets into a dictionary
all_sheets = pd.read_excel(excel_file, sheet_name=None)

# Process each sheet
modified_sheets = {}

for sheet_name, df in all_sheets.items():
    # Identify the attribute column (excluding datetime)
    attribute = df.columns[1]  # Second column contains the attribute
    
    # Rename the attribute column to "modified_{attribute}"
    modified_attribute = f"mod_{attribute}"
    
    # Apply transformation: modified_data = original_data * trend + noise
    df[modified_attribute] = df[attribute] + (df.index + 1) * trend + noise
    
    # Keep only the datetime column and the modified attribute
    df = df[['datetime', modified_attribute]]
    
    # Store the modified DataFrame
    modified_sheets[sheet_name] = df

# Save the modified data into a new Excel file
# modified_excel_file = "PdM_modified_machine_time_series.xlsx"
modified_excel_file = "Chronos_PdM_modified_machine_time_series.xlsx"

with pd.ExcelWriter(modified_excel_file) as writer:
    for sheet_name, df in modified_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("Data transformation complete.")

Data transformation complete.


In [ ]:
import pandas as pd

excel_path = 'Chronos_PdM_modified_slope_calc.xlsx'

excel_file = pd.ExcelFile(excel_path)

sheet_names = excel_file.sheet_names

new_excel = pd.ExcelWriter('PdM_machine_time_series.xlsx')

for sheet_name in sheet_names:
    df = excel_file.parse(sheet_name)

    last_100 = df.tail(100)

    last_100.to_excel(new_excel, sheet_name=sheet_name, index=False)

new_excel.close()

In [ ]:
## Negative Slope

# Define transformation parameters
trend = -2
noise = 6

# Load the Excel file
excel_file = "PdM_machine_time_series.xlsx"

# Read all sheets into a dictionary
all_sheets = pd.read_excel(excel_file, sheet_name=None)

# Process each sheet
modified_sheets = {}

for sheet_name, df in all_sheets.items():
    # Identify the attribute column (excluding datetime)
    attribute = df.columns[1]  # Second column contains the attribute
    
    # Rename the attribute column to "modified_{attribute}"
    modified_attribute = f"mod_{attribute}"
    
    # Apply transformation: modified_data = original_data * trend + noise
    df[modified_attribute] = df[attribute] + (df.index + 1) * trend + noise
    
    # Keep only the datetime column and the modified attribute
    df = df[['datetime', modified_attribute]]
    
    # Store the modified DataFrame
    modified_sheets[sheet_name] = df

# Save the modified data into a new Excel file
modified_excel_file = "neg_modified_machine_time_series.xlsx"

with pd.ExcelWriter(modified_excel_file) as writer:
    for sheet_name, df in modified_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("Data transformation complete. Negative Modified Excel file saved as 'neg_modified_machine_time_series.xlsx'.")

### 20 time series for forecasting, with 200, 500, 2000, 5000, and all data (8761) for each of 4 variables from machine 1

In [2]:
import pandas as pd

# Load the dataset
file_path = "PdM_telemetry.csv"  # Replace with actual file path
df = pd.read_csv(file_path)

# Ensure datetime is correctly parsed
df['datetime'] = pd.to_datetime(df['datetime'])

# Filter data for machine_id = 1
machine_df = df[df['machineID'] == 1].sort_values(by="datetime")

# Variables to extract
variables = ["volt", "rotate", "pressure", "vibration"]
sample_sizes = [200, 500, 2000, 5000, len(machine_df)]  # Including full dataset

# Dictionary to store extracted subsets
extracted_data = {}

# Extract subsets for each variable
for var in variables:
    for size in sample_sizes:
        subset_name = f"{var}_{size}"
        extracted_data[subset_name] = machine_df[['datetime', var]].iloc[:size]

# Save extracted subsets to an Excel file with multiple sheets
output_file = "PdM_forecasting.xlsx"

with pd.ExcelWriter(output_file) as writer:
    for subset_name, df_subset in extracted_data.items():
        df_subset.to_excel(writer, sheet_name=subset_name, index=False)

print("Data extraction complete. Saved as 'PdM_forecasting.xlsx'.")

/var/folders/fh/x42m5xbn25d25fy3bskk3jkm0000gn/T/ipykernel_32733/3650173335.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['datetime'] = pd.to_datetime(df['datetime'])


Data extraction complete. Saved as 'PdM_forecasting.xlsx'.
